<a href="https://colab.research.google.com/github/acb100cias/tutoriales_computo_fc/blob/master/PyMC2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymc

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm

In [ ]:
test_scores=pd.read_csv(pm.get_data('test_scores.csv'),index_col=0)

In [ ]:
test_scores

In [ ]:
test_scores['score'].hist()

In [ ]:
X=test_scores.dropna().astype(float)

In [ ]:
X

In [ ]:
Y=X.pop('score')

In [ ]:
X-=X.mean()
X/=X.std()

In [ ]:
N,D=X.shape

In [ ]:
X

In [ ]:
D0=int(D/2)

In [ ]:
import pytensor.tensor as pt

In [ ]:
with pm.Model(coords={'predictors':X.columns.values}) as test_score_model:
  sigma=pm.HalfNormal("sigma",25)
  tau=pm.HalfStudentT('tau',2,D0/(D-D0)*sigma/np.sqrt(N))
  lam=pm.HalfStudentT('lam',5,dims='predictors')
  c2=pm.InverseGamma('c2',1,1)
  z=pm.Normal('z',0.0,1.0,dims='predictors')
  beta=pm.Deterministic('beta',z*tau*lam*pt.sqrt(c2/(c2+tau**2*lam**2)),dims='predictors')
  beta0=pm.Normal('beta0',100,25.0)
  scores=pm.Normal('scores',beta0+pt.dot(X.values,beta),sigma,observed=Y.values)

In [ ]:
pm.model_to_graphviz(test_score_model)

In [ ]:
with test_score_model:
  prior_samples=pm.sample_prior_predictive(100)


In [ ]:
az.plot_dist(test_scores['score'].values,kind='hist',color='C1',hist_kwargs={'alpha':0.6},label='observed')
az.plot_dist(prior_samples.prior_predictive['scores'],kind='hist',hist_kwargs={'alpha':0.6},label='simulated')
plt.xticks(rotation=45)

In [ ]:
with test_score_model:
  idata=pm.sample(1000,tune=2000,random_seed=42)

In [ ]:
with test_score_model:
  idata=pm.sample(1000,tune=2000,random_seed=42,target_accept=0.99)

In [ ]:
az.plot_trace(idata,var_names=['tau','sigma','c2'])

In [ ]:
az.plot_energy(idata)

In [ ]:
az.plot_forest(idata,var_names=['beta'],combined=True,hdi_prob=0.95,r_hat=True)